In [1]:
import wandb 
wandb.login()

wandb: Currently logged in as: nxhoang562 (nxhoang). Use `wandb login --relogin` to force relogin


True

# Libraries

In [2]:
import os

import argparse
import sys
import torch
import yaml
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from dataset import make_dataset, make_dataloader #dataset containing noise 

from orignial_dataset import make_dataset as original_make_dataset
from orignial_dataset import make_dataloader as original_make_dataloader
from sklearn.model_selection import train_test_split


from tqdm import tqdm
from metrics import compute_pck_pckh
import numpy as np
from torch import nn
from torchsummary import summary

# sys.path.append("/home/nxhoang/Work/HPE")
# # from src.denoise.model import AutoEncoder
# # from src.denoise.train import Trainer as Denoise_Trainer
# # from src.model.Denoise_Fdcnn import CombinedModel
# # from src.keypoint_detection.models.fdcnn import CNN
# # from src.keypoint_detection.trainer import Trainer as Estimastor_Trainer

# Model


In [3]:
#Some codes are adopted from cite dynamic cnn 

class Dynamic_conv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, bias=False, n_basis_kernels=4,
                 temperature=31, pool_dim='freq'):
        super(Dynamic_conv2d, self).__init__()

        self.in_planes = in_planes
        self.out_planes = out_planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.pool_dim = pool_dim

        self.n_basis_kernels = n_basis_kernels
        self.attention = attention2d(in_planes, self.kernel_size, self.stride, self.padding, n_basis_kernels,
                                     temperature, pool_dim)

        self.weight = nn.Parameter(torch.randn(n_basis_kernels, out_planes, in_planes, self.kernel_size, self.kernel_size),
                                   requires_grad=True)

        if bias:
            self.bias = nn.Parameter(torch.Tensor(n_basis_kernels, out_planes))
        else:
            self.bias = None

        for i in range(self.n_basis_kernels):
            nn.init.kaiming_normal_(self.weight[i])

    def forward(self, x): #x size : [bs, in_chan, frames, freqs]   -> new: [bs, in_chan, freqs, frames]
        if self.pool_dim in ['freq', 'chan']:
            # softmax_attention = self.attention(x).unsqueeze(2).unsqueeze(4)    # size : [bs, n_ker, 1, frames, 1]
            softmax_attention = self.attention(x).unsqueeze(2).unsqueeze(3)    # size : [bs, n_ker, 1,1,frames]
        elif self.pool_dim == 'time':
            softmax_attention = self.attention(x).unsqueeze(2).unsqueeze(4)    # size : [bs, n_ker, 1, freqs, 1]
        elif self.pool_dim == 'both':
            softmax_attention = self.attention(x).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)    # size : [bs, n_ker, 1, 1, 1]

        batch_size = x.size(0)

        aggregate_weight = self.weight.view(-1, self.in_planes, self.kernel_size, self.kernel_size) # size : [n_ker * out_chan, in_chan]

        if self.bias is not None:
            aggregate_bias = self.bias.view(-1)
            output = F.conv2d(x, weight=aggregate_weight, bias=aggregate_bias, stride=self.stride, padding=self.padding)
        else:
            output = F.conv2d(x, weight=aggregate_weight, bias=None, stride=self.stride, padding=self.padding)
            # output size : [bs, n_ker * out_chan, frames, freqs]

        output = output.view(batch_size, self.n_basis_kernels, self.out_planes, output.size(-2), output.size(-1))
        # output size : [bs, n_ker, out_chan, frames, freqs]

        if self.pool_dim in ['freq', 'chan']:
            assert softmax_attention.shape[-1] == output.shape[-1]
        elif self.pool_dim == 'time':
            assert softmax_attention.shape[-2] == output.shape[-2]

        output = torch.sum(output * softmax_attention, dim=1)  # output size : [bs, out_chan, frames, freqs]

        return output


class attention2d(nn.Module):
    def __init__(self, in_planes, kernel_size, stride, padding, n_basis_kernels, temperature, pool_dim):
        super(attention2d, self).__init__()
        self.pool_dim = pool_dim
        self.temperature = temperature

        hidden_planes = int(in_planes / 4)

        if hidden_planes < 4:
            hidden_planes = 4

        if not pool_dim == 'both':
            self.conv1d1 = nn.Conv1d(in_planes, hidden_planes, kernel_size, stride=stride, padding=padding, bias=False)
            self.bn = nn.BatchNorm1d(hidden_planes)
            self.relu = nn.ReLU(inplace=True)
            self.conv1d2 = nn.Conv1d(hidden_planes, n_basis_kernels, 1, bias=True)
            for m in self.modules():
                if isinstance(m, nn.Conv1d):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                if isinstance(m, nn.BatchNorm1d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
        else:
            self.fc1 = nn.Linear(in_planes, hidden_planes)
            self.relu = nn.ReLU(inplace=True)
            self.fc2 = nn.Linear(hidden_planes, n_basis_kernels)


    def forward(self, x): #CSI size : [bs, chan, freqs, frames] 
        if self.pool_dim == 'freq':
            x = torch.mean(x, dim=2)  #x size : [bs, chan, frames] 
        elif self.pool_dim == 'time':
            x = torch.mean(x, dim=3)  #x size : [bs, chan, freqs]
        elif self.pool_dim == 'both':
            # x = torch.mean(torch.mean(x, dim=2), dim=1)  #x size : [bs, chan]
            x = F.adaptive_avg_pool2d(x, (1, 1)).squeeze(-1).squeeze(-1)
        elif self.pool_dim == 'chan':
            x = torch.mean(x, dim=1)  #x size : [bs, freqs, frames]

        if not self.pool_dim == 'both':
            x = self.conv1d1(x)               #x size : [bs, hid_chan, frames]
            x = self.bn(x)
            x = self.relu(x)
            x = self.conv1d2(x)               #x size : [bs, n_ker, frames]
        else:
            x = self.fc1(x)               #x size : [bs, hid_chan]
            x = self.relu(x)
            x = self.fc2(x)               #x size : [bs, n_ker]

        return F.softmax(x / self.temperature, 1)

In [4]:
# DENOISER 

# Doi Batch Norm len truoc 


#d_hidden, dropout_ae
class Encoder(nn.Module):
    def __init__(self, shape_data, n_kernels, kernel_size, maxpooling):
        super().__init__()
        self.shape_data = shape_data
        self.n_kernels = n_kernels
        self.kernel_size = kernel_size
        self.maxpooling = maxpooling 

      # input -> conv2d -> batchnorm -> maxpool2d ->conv2d -> batchnorm -> maxpool     
       # layer1
        self.conv1 = nn.Conv2d(
                                in_channels=self.shape_data[0],
                                out_channels=self.n_kernels[0],
                                kernel_size=self.kernel_size[0],
                                padding=1
                               )
        self.atv1 = nn.ReLU()
        self.batchNorm1 = nn.BatchNorm2d(num_features=self.n_kernels[0])
        self.pool1 = nn.MaxPool2d(kernel_size=self.maxpooling[0])     

        # layer 2 
        self.conv2 = nn.Conv2d(
                                in_channels=self.n_kernels[0],
                                out_channels=self.n_kernels[1],
                                kernel_size=self.kernel_size[1],
                                padding=1
                                )
        self.atv2 = nn.ReLU()
        self.batchNorm2 = nn.BatchNorm2d(num_features=self.n_kernels[1])
        self.pool2 = nn.MaxPool2d(kernel_size=self.maxpooling[1]) 
       
        # height and weight of the output shape of conv1 
        self.h_conv1 = int((self.shape_data[1] - self.kernel_size[0] + 3 )) # padding = 1
        self.w_conv1 = int((self.shape_data[2] - self.kernel_size[0] + 3 ))      
        
        # output of max-pooling 1
        self.h_pool1 = int((self.h_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)
        # stride of max-pooling = maxpooling 
        self.w_pool1 = int((self.w_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1) 

        # height and weight of the output shape of conv2 
        self.h_conv2 = int(self.h_pool1 - self.kernel_size[1] + 3 ) # padding = 1
        self.w_conv2 = int(self.w_pool1 - self.kernel_size[1] + 3 )      
        
        #output of max-pooling 2
        self.h_pool2 = int((self.h_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)
        self.w_pool2 = int((self.w_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1) 
        
        # # layer3
        # self.conv3 = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, padding=0)

    def forward(self, x):
        # x = self.pool1(self.batchNorm1(self.relu1(self.conv1(x))))
        x = self.pool1(self.atv1(self.batchNorm1(self.conv1(x))))
        x = self.pool2(self.atv2(self.batchNorm2(self.conv2(x))))

       
        # x = self.pool3(self.relu3(self.conv3(x)))
        return x


class Decoder(nn.Module):
    def __init__(self, shape_data, n_kernels, kernel_size, maxpooling):
        super().__init__()
        self.shape_data = shape_data
        self.n_kernels = n_kernels
        self.kernel_size = kernel_size
        self.maxpooling = maxpooling 

        self.h_conv1 = int((self.shape_data[1] - self.kernel_size[0] + 3 ))
        self.w_conv1 = int((self.shape_data[2] - self.kernel_size[0] + 3 ))   
        self.h_pool1 = int((self.h_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1)
        self.w_pool1 = int((self.w_conv1 - self.maxpooling[0])/self.maxpooling[0] + 1) 
        
        self.h_conv2 = int(self.h_pool1 - self.kernel_size[1] + 3 )
        self.w_conv2 = int(self.w_pool1 - self.kernel_size[1] + 3)    
        
        self.h_pool2 = int((self.h_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1)
        self.w_pool2 = int((self.w_conv2 - self.maxpooling[1])/self.maxpooling[1] + 1) 
        
        # Layer 3 ~ hidden layer
        self.conv3 = nn.Conv2d(in_channels=self.n_kernels[1],
                               out_channels=self.n_kernels[1],
                               kernel_size=self.kernel_size[1],
                               padding=1
                               )
        self.atv3 = nn.ReLU()
        self.batchNorm3 = nn.BatchNorm2d(num_features=self.n_kernels[1])
        # Upsampling:  desired_output_size = (32, 32)  # Upsample to 32x32
        # upsample_layer = nn.Upsample(size=desired_output_size, mode='bilinear', align_corners=False)
        self.up_pool3 = nn.Upsample(size =(self.h_conv2, self.w_conv2))

        # Layer 4
        self.conv4 = nn.Conv2d(in_channels=self.n_kernels[1],
                               out_channels=self.n_kernels[0],
                               kernel_size=self.kernel_size[0],
                               padding=1 
                                )
        self.atv4 = nn.ReLU()
        self.batchNorm4 = nn.BatchNorm2d(num_features=self.n_kernels[0])

        self.up_pool4 = nn.Upsample(size = (self.h_conv1, self.w_conv1) )

        # Layer 5  = output 
        self.conv5 = nn.Conv2d(in_channels=self.n_kernels[0],
                               out_channels=self.shape_data[0],
                               kernel_size=self.kernel_size[0],
                               padding=1 
                                )
        self.atv5 = nn.ReLU()

    def forward(self, x):
        x = self.up_pool3(self.batchNorm3(self.atv3(self.conv3(x))))
        x = self.up_pool4(self.batchNorm4(self.atv4(self.conv4(x))))
        x = self.atv5(self.conv5(x))

        return x


class Denoiser(nn.Module):
    def __init__(self, shape_data, n_kernels, kernel_size, maxpooling):
        super().__init__()
        self.shape_data = shape_data
        self.n_kernels = n_kernels
        self.kernel_size = kernel_size
        self.maxpooling = maxpooling
        self.encoder = Encoder(self.shape_data,  self.n_kernels, self.kernel_size, self.maxpooling)
        self.decoder = Decoder(self.shape_data,  self.n_kernels, self.kernel_size, self.maxpooling)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


# if __name__ == '__main__':
#     test_model = AutoEncoder().to("cuda")

#     # ummary(test_model, input_size=(3, 32,136), batch_size=16, device="cuda")
#     pass

# model = AutoEncoder(shape_data = (3,32,136), n_kernels = [64,4], kernel_size = [3,3], maxpooling = [2,2]).to("cuda")
model = Denoiser(shape_data = (3,136,32), n_kernels = [26,183], kernel_size = [3,3], maxpooling = [2,2]).to("cuda")
summary(model, (3, 136, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 26, 136, 32]             728
       BatchNorm2d-2          [-1, 26, 136, 32]              52
              ReLU-3          [-1, 26, 136, 32]               0
         MaxPool2d-4           [-1, 26, 68, 16]               0
            Conv2d-5          [-1, 183, 68, 16]          43,005
       BatchNorm2d-6          [-1, 183, 68, 16]             366
              ReLU-7          [-1, 183, 68, 16]               0
         MaxPool2d-8           [-1, 183, 34, 8]               0
           Encoder-9           [-1, 183, 34, 8]               0
           Conv2d-10           [-1, 183, 34, 8]         301,584
             ReLU-11           [-1, 183, 34, 8]               0
      BatchNorm2d-12           [-1, 183, 34, 8]             366
         Upsample-13          [-1, 183, 68, 16]               0
           Conv2d-14           [-1, 26,

In [5]:
# Keypoint Estimator 

class Estimator(nn.Module):

    
    def __init__(self, shape_data,  
                dropout = None,
                kernel_size = [3, 3, 3,3,3],
                n_kernels =[8, 8, 8,8,8],   
                num_layers = 2,  
                pad = [1, 1, 1,1,1], #fix la 1,k doi 
                stride = [1, 1, 1,1,1], #fix la 1, k doi                              
                maxPooling = [2,2,2,2,2],
                n_basis_kernels = 5, 
                temperature = 31,
                d_linear = 128,
                pool_dim = 'time'): 
        super(Estimator, self).__init__()
        
        self.num_layers  = num_layers # no. dcnn layers 
        # self.est_n_input_ch = est_n_input_ch  # no. channels
        self.shape_data = shape_data
        self.dropout = dropout
        self.kernel_size = kernel_size # kernel size
        self.n_kernels = n_kernels  # no. filters for each conv layer
        self.pad = pad
        self.stride = stride
        self.maxPooling = maxPooling   # pooling dimensions for each poooling layers 
        self.n_basis_kernels = n_basis_kernels  # no. kernels
        
        self.temperature = temperature
        self.pool_dim = pool_dim # dimension for pooling
        self.d_linear = d_linear
        # self.est_n_filt_last = est_kernel_size[-1]

        
        self.h_conv = [0]*self.num_layers
        self.w_conv = [0]*self.num_layers
        self.h_pool = [0]*self.num_layers
        self.w_pool = [0]*self.num_layers
        in_dim = [0]*self.num_layers
        out_dim = [0]*self.num_layers


        cnn = nn.Sequential()

        def conv(i, dropout=None):
            in_dim[i] = self.shape_data[0] if i == 0 else self.n_kernels[i-1] #kenh dau vao cua conv
            out_dim[i] = self.n_kernels[i] #kenh dau ra cua conv 

            cnn.add_module( "conv{0}".format(i),
                            Dynamic_conv2d(in_dim[i],
                                           out_dim[i],
                                           self.kernel_size[i],
                                           self.stride[i],
                                           self.pad[i], 
                                           n_basis_kernels = self.n_basis_kernels, 
                                           temperature=self.temperature, 
                                           pool_dim=self.pool_dim)
                            ) 

            
            cnn.add_module("batchNorm{0}".format(i), 
                           nn.BatchNorm2d(out_dim[i], momentum = 0.99)
                           )
                
            cnn.add_module("activ{0}".format(i), 
                           nn.ReLU())
            
            if dropout is not None: 
                cnn.add_module("dropout{0}".format(i),
                               nn.Dropout(dropout))
                 
            cnn.add_module("pooling{0}".format(i), 
                           nn.MaxPool2d(self.maxPooling[i]))
            
            if i == 0:
                self.h_conv[0] = int((self.shape_data[1] - self.kernel_size[0] + 3 ))
                self.w_conv[0] = int((self.shape_data[2] - self.kernel_size[0] + 3)) 

                self.h_pool[0] = int((self.h_conv[0] - self.maxPooling[0])/self.maxPooling[0] + 1)
                self.w_pool[0] = int((self.w_conv[0] - self.maxPooling[0])/self.maxPooling[0] + 1)
            
            else:
                self.h_conv[i] = int((self.h_pool[i-1] - self.kernel_size[i] + 3 ))
                self.w_conv[i] = int((self.w_pool[i-1] - self.kernel_size[i] + 3 ))
                
                self.h_pool[i] = int((self.h_conv[i] - self.maxPooling[i])/self.maxPooling[i] + 1)
                self.w_pool[i] = int((self.w_conv[i] - self.maxPooling[i])/self.maxPooling[i] + 1)
        
        
        for i in range(self.num_layers):  
            conv(i, dropout= self.dropout)       
        self.cnn = cnn
        
        self.fc1 = nn.Linear(in_features= out_dim[self.num_layers-1]*self.h_pool[self.num_layers-1]*self.w_pool[self.num_layers-1], out_features= self.d_linear)
        self.batch_norm = nn.BatchNorm1d(num_features=self.d_linear)
        self.fc2 = nn.Linear(in_features=self.d_linear, out_features=34)
        #self.drop = nn.Dropout(p=dropout)

    def forward(self, x):  # x size : [batch, channel, frames, freqs] 32, 3, 10,114   ```` 32,136 
        batch = x.shape[0]
        # print(x.shape)
        
        x = self.cnn(x)

        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = self.fc1(x)
        x = self.batch_norm(x)
        x = nn.functional.relu(x)
        # x = self.drop(x)
        x = self.fc2(x)
        x = x.reshape(batch, 17, 2)

        return x


model = Estimator(shape_data = (3,136,32)).to("cuda")

summary(model, (3, 136, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 4, 136]              36
       BatchNorm1d-2               [-1, 4, 136]               8
              ReLU-3               [-1, 4, 136]               0
            Conv1d-4               [-1, 5, 136]              25
       attention2d-5               [-1, 5, 136]               0
    Dynamic_conv2d-6           [-1, 8, 136, 32]           1,080
       BatchNorm2d-7           [-1, 8, 136, 32]              16
              ReLU-8           [-1, 8, 136, 32]               0
         MaxPool2d-9            [-1, 8, 68, 16]               0
           Conv1d-10                [-1, 4, 68]              96
      BatchNorm1d-11                [-1, 4, 68]               8
             ReLU-12                [-1, 4, 68]               0
           Conv1d-13                [-1, 5, 68]              25
      attention2d-14                [-1

In [6]:

class CombinedModel(nn.Module):
    def __init__(self, denoiser, predictor):
        super(CombinedModel, self).__init__()
        # Denoiser
        self.denoiser = denoiser
        # FD CNN
        self.predictor = predictor

    def forward(self, x):
        with torch.no_grad():
            encoded = self.denoiser(x)
        output = self.predictor(encoded)
        return output



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

AE = Denoiser(shape_data = (3,136,32), n_kernels = [26,183], kernel_size = [3,3], maxpooling = [2,3])

CNN_input_shape = (AE.encoder.n_kernels[-1], AE.encoder.h_pool2, AE.encoder.w_pool2)

CNN = Estimator(shape_data=CNN_input_shape, 
                              dropout=0.1,
                              n_basis_kernels=16,
                              kernel_size=[2,2,2,2,2],
                              n_kernels=[16,16,16,16,16],
                              num_layers= 5,
                              pad=[1,1,1,1,1,1],
                              stride = [1,1,1,1,1],
                              maxPooling= (2,2,2,2,2),
                              temperature=30, 
                              pool_dim='time',
                              d_linear = 256
                              )

model = CombinedModel(AE.encoder, CNN)
model = model.to(device)
summary(model, (3, 136, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 26, 136, 32]             728
       BatchNorm2d-2          [-1, 26, 136, 32]              52
              ReLU-3          [-1, 26, 136, 32]               0
         MaxPool2d-4           [-1, 26, 68, 16]               0
            Conv2d-5          [-1, 183, 68, 16]          43,005
       BatchNorm2d-6          [-1, 183, 68, 16]             366
              ReLU-7          [-1, 183, 68, 16]               0
         MaxPool2d-8           [-1, 183, 22, 5]               0
           Encoder-9           [-1, 183, 22, 5]               0
           Conv1d-10               [-1, 45, 23]          16,470
      BatchNorm1d-11               [-1, 45, 23]              90
             ReLU-12               [-1, 45, 23]               0
           Conv1d-13               [-1, 16, 23]             736
      attention2d-14               [-1,

# Trainer

In [7]:
class Denoise_Trainer:
    def __init__(self, model, train_loader, val_loader, test_loader, criterion, optimizer,
                 scheduler, model_save_path="checkpoints"):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.model_save_path = model_save_path
        self.metric = dict()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.train_epoch_idx = 0
        self.val_epoch_idx = 0

    def train_epoch(self):
        print(f"Training Denoiser. Epoch: {self.train_epoch_idx}")
        self.metric[self.train_epoch_idx] = dict()
        self.model.train()
        losses = []
        
        for iter, (data, label) in enumerate(self.train_loader):
            data = data.to(self.device)
            label = label.to(self.device)
            predict = self.model(data)
            loss = self.criterion(predict, label)

            losses.append(loss.item())
            print(f"Iter {iter + self.train_epoch_idx * len(self.train_loader)}: MSE - {losses[-1]}")

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        self.metric[self.train_epoch_idx]["denoiser_train_loss"] = sum(losses) / len(losses)
        wandb.log({"denoiser_train_loss": sum(losses) / len(losses) })
        self.train_epoch_idx += 1

    def val_epoch(self):
        print(f"Validating Denoiser. Epoch: {self.val_epoch_idx}")
        self.metric[self.train_epoch_idx] = dict()
        self.model.eval()
        losses = []
        for data, label in self.val_loader:
            data = data.to(self.device)
            label = label.to(self.device)
            with torch.no_grad():
                predict = self.model(data)
            loss = self.criterion(predict, label)

            losses.append(loss.item())

        self.metric[self.val_epoch_idx]["denoiser_val_loss"] = sum(losses) / len(losses)
        wandb.log({"denoiser_val_loss": sum(losses) / len(losses) })
        # print(f"Val loss: {sum(losses) / len(losses)}")
        if self.val_epoch_idx == 0 or \
                self.metric[self.val_epoch_idx]["denoiser_val_loss"] < self.metric[self.val_epoch_idx - 1]["denoiser_val_loss"]:
            self.save_model()
            print(f"Denoiser Model is saved at epoch {self.val_epoch_idx}")
        self.val_epoch_idx += 1

    def test_epoch(self):
        print(f"Testing Denoiser")
        # self.model.load_state_dict(torch.load("/home/nxhoang/Work/HPE/src/model/checkpoints/model_best.pth")['model_state_dict'])
        self.model.eval()
        losses = []
        for data, label in tqdm(self.test_loader):
            data = data.to(self.device)
            label = label.to(self.device)
            with torch.no_grad():
                predict = self.model(data)
            loss = self.criterion(predict, label)

            losses.append(loss.item())

        self.metric["test"] = sum(losses) / len(losses)
        wandb.log({"Denoiser test result": self.metric["test"]})
        # print(f"Denoiser test result: {self.metric['test']}")

    def save_model(self):
        state_dict = dict()
        state_dict["model_state_dict"] = self.model.state_dict()
        state_dict["optimizer_state_dict"] = self.optimizer.state_dict()
        state_dict["train_loss_history"] = torch.tensor([self.metric[i]["denoiser_train_loss"] for i in range(self.train_epoch_idx)])
        state_dict["val_loss_history"] = torch.tensor([self.metric[i]["denoiser_val_loss"] for i in range(self.val_epoch_idx)])

        save_path = os.path.join(self.model_save_path, f"model_best.pth")
        torch.save(state_dict, save_path)


class Estimastor_Trainer:
    def __init__(self, model, train_loader, val_loader, test_loader, criterion, optimizer,
                 scheduler, model_save_path="checkpoints"):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.model_save_path = model_save_path
        self.metric = dict()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.train_epoch_idx = 0
        self.val_epoch_idx = 0
        self.min_val_loss = 9999

    def train_epoch(self):
        print(f"Training Estimator.  Epoch {self.train_epoch_idx}")
        self.metric[self.train_epoch_idx] = dict()
        self.model.train()
        losses = []

        for iter, (data, gt) in enumerate(self.train_loader):
            # print(data.shape)
            data = np.transpose(data, (0, 1, 3, 2))
            # print('data_shape', data.shape)
            data = data.to(self.device)
            confidence = gt[:, :, 2:].to(self.device)
            # print('shape confidence', confidence.shape)
            label = gt[:, :, 0:2].to(self.device)
            # print('shape label', label.shape)
            predict = self.model(data)
            # print('shape predict', predict.shape)

            loss = self.criterion(torch.mul(predict, confidence), torch.mul(label, confidence)) / 32

            losses.append(loss.item())
            # print(f"Iter {iter + self.train_epoch_idx * len(self.train_loader)}: MSE - {losses[-1]}")

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        self.scheduler.step()

        self.metric[self.train_epoch_idx]["train_loss"] = sum(losses) / len(losses)
        wandb.log({"Estimator Training Loss": self.metric[self.train_epoch_idx]["train_loss"]})   
        self.train_epoch_idx += 1

    def val_epoch(self):
        print(f"Validating Estimator. Epoch {self.val_epoch_idx}")
        self.metric[self.train_epoch_idx] = dict()
        self.model.eval()

        pck_50_iter = []
        pck_40_iter = []
        pck_30_iter = []
        pck_20_iter = []
        pck_10_iter = []
        pck_5_iter = []

        losses = []
        for data, gt in self.val_loader:
            data = np.transpose(data, (0, 1, 3, 2)) # [batch size, channels, freq, time]
            data = data.to(self.device)
            label = gt[:, :, 0:2].to(self.device)
            confidence = gt[:, :, 2:].to(self.device)
            with torch.no_grad():
                predict = self.model(data)
            loss = self.criterion(torch.mul(confidence, predict), torch.mul(confidence, label))

            losses.append(loss.item())

            predict = predict.cpu()
            label = label.cpu()

            predict = torch.transpose(predict, 1, 2)
            label = torch.transpose(label, 1, 2)

            pck_50_iter.append(compute_pck_pckh(predict, label, 0.5))
            pck_40_iter.append(compute_pck_pckh(predict, label, 0.4))
            pck_30_iter.append(compute_pck_pckh(predict, label, 0.3))
            pck_20_iter.append(compute_pck_pckh(predict, label, 0.2))
            pck_10_iter.append(compute_pck_pckh(predict, label, 0.1))
            pck_5_iter.append(compute_pck_pckh(predict, label, 0.05))

        self.metric[self.val_epoch_idx]["val loss"] = sum(losses) / len(losses)
        self.metric[self.val_epoch_idx]["pck_50"] = sum(pck_50_iter) / len(pck_50_iter)
        self.metric[self.val_epoch_idx]["pck_40"] = sum(pck_40_iter) / len(pck_40_iter)
        self.metric[self.val_epoch_idx]["pck_30"] = sum(pck_30_iter) / len(pck_30_iter)
        self.metric[self.val_epoch_idx]["pck_20"] = sum(pck_20_iter) / len(pck_20_iter)
        self.metric[self.val_epoch_idx]["pck_10"] = sum(pck_10_iter) / len(pck_10_iter)
        self.metric[self.val_epoch_idx]["pck_5"] = sum(pck_5_iter) / len(pck_5_iter)

        loss_avg = sum(losses) / len(losses)
        # print(f"Val loss: {loss_avg}")
        # print("pck_50: ", sum(pck_50_iter) / len(pck_50_iter))
        # print("pck_40: ", sum(pck_40_iter) / len(pck_40_iter))
        # print("pck_30: ", sum(pck_30_iter) / len(pck_30_iter))
        # print("pck_20: ", sum(pck_20_iter) / len(pck_20_iter))
        # print("pck_10: ", sum(pck_10_iter) / len(pck_10_iter))
        # print("pck_5: ", sum(pck_5_iter) / len(pck_5_iter))

        wandb.log({'estimator_val_loss': loss_avg})
        
        if loss_avg < 0.4:
            wandb.alert(
                title='Xuan Hoang just found the best loss_val',
                text=f'Val Loss{loss_avg} is below 0.4',
            )
            print('Alert triggered') 



        wandb.log({'pck_50 Val': sum(pck_50_iter) / len(pck_50_iter)})
        wandb.log({'pck_40 Val': sum(pck_40_iter) / len(pck_40_iter)})
        wandb.log({'pck_30 Val': sum(pck_30_iter) / len(pck_30_iter)})
        wandb.log({'pck_20 Val': sum(pck_20_iter) / len(pck_20_iter)})
        wandb.log({'pck_10 Val': sum(pck_10_iter) / len(pck_10_iter)})
        wandb.log({'pck_5 Val': sum(pck_5_iter) / len(pck_5_iter)})
     

        # logs = "Val loss: " + str(sum(losses) / len(losses)) + ", "
        # logs += "pck_50: " + str(sum(pck_50_iter) / len(pck_50_iter)) + ", "
        # logs += "pck_40: " + str(sum(pck_40_iter) / len(pck_40_iter)) + ", "
        # logs += "pck_30: " + str(sum(pck_30_iter) / len(pck_30_iter)) + ", "
        # logs += "pck_20: " + str(sum(pck_20_iter) / len(pck_20_iter)) + ", "
        # logs += "pck_10: " + str(sum(pck_10_iter) / len(pck_10_iter)) + ", "
        # logs += "pck_5: " + str(sum(pck_5_iter) / len(pck_5_iter)) + "\n"
        
        # with open("/home/nxhoang/Work/HPE/src/model/logs/combined_model.txt", "a") as f:
        
        #     f.write(logs)

        if self.val_epoch_idx == 0 or \
                self.metric[self.val_epoch_idx]["val loss"] < self.min_val_loss:
            self.min_val_loss = self.metric[self.val_epoch_idx]["val loss"]
            self.save_model()
            print(f"Estimator Model is saved at epoch {self.val_epoch_idx}")
        self.val_epoch_idx += 1

    def test_epoch(self):
        print(f"Estmator Testing")
        self.model.load_state_dict(torch.load("/home/nxhoang/Work/HPE/src/model/checkpoints/model_best.pth")['model_state_dict'])
        self.model.eval()
        pck_50_iter = []
        pck_40_iter = []
        pck_30_iter = []
        pck_20_iter = []
        pck_10_iter = []
        pck_5_iter = []
        losses = []
        for data, gt in tqdm(self.test_loader):
            data = np.transpose(data, (0, 1, 3, 2))  # [batch size, channels, freq, time]
            data = data.to(self.device)
            label = gt[:, :, 0:2].to(self.device)
            confidence = gt[:, :, 2:].to(self.device)
            with torch.no_grad():
                predict = self.model(data)
            loss = self.criterion(torch.mul(predict, confidence), torch.mul(label, confidence))

            losses.append(loss.item())

            predict = predict.cpu()
            label = label.cpu()
            predict = torch.transpose(predict, 1, 2)
            label = torch.transpose(label, 1, 2)

            pck_50_iter.append(compute_pck_pckh(predict, label, 0.5))
            pck_40_iter.append(compute_pck_pckh(predict, label, 0.4))
            pck_30_iter.append(compute_pck_pckh(predict, label, 0.3))
            pck_20_iter.append(compute_pck_pckh(predict, label, 0.2))
            pck_10_iter.append(compute_pck_pckh(predict, label, 0.1))
            pck_5_iter.append(compute_pck_pckh(predict, label, 0.05))

        self.metric["test"] = dict()
        self.metric["test"]["loss"] = sum(losses) / len(losses)
        self.metric["test"]["pck_50"] = sum(pck_50_iter) / len(pck_50_iter)
        self.metric["test"]["pck_40"] = sum(pck_40_iter) / len(pck_40_iter)
        self.metric["test"]["pck_30"] = sum(pck_30_iter) / len(pck_30_iter)
        self.metric["test"]["pck_20"] = sum(pck_20_iter) / len(pck_20_iter)
        self.metric["test"]["pck_10"] = sum(pck_10_iter) / len(pck_10_iter)
        self.metric["test"]["pck_5"] = sum(pck_5_iter) / len(pck_5_iter)

        # wandb.log({'Estimator Tes': loss_avg})
        wandb.log({'pck_50': sum(pck_50_iter) / len(pck_50_iter)})
        wandb.log({'pck_40': sum(pck_40_iter) / len(pck_40_iter)})
        wandb.log({'pck_30': sum(pck_30_iter) / len(pck_30_iter)})
        wandb.log({'pck_20': sum(pck_20_iter) / len(pck_20_iter)})
        wandb.log({'pck_10': sum(pck_10_iter) / len(pck_10_iter)})
        wandb.log({'pck_5': sum(pck_5_iter) / len(pck_5_iter)})

        # print(f"Test result: {self.metric['test']}")
        # print("pck_50: ", sum(pck_50_iter) / len(pck_50_iter))
        # print("pck_40: ", sum(pck_40_iter) / len(pck_40_iter))
        # print("pck_30: ", sum(pck_30_iter) / len(pck_30_iter))
        # print("pck_20: ", sum(pck_20_iter) / len(pck_20_iter))
        # print("pck_10: ", sum(pck_10_iter) / len(pck_10_iter))
        # print("pck_5: ", sum(pck_5_iter) / len(pck_5_iter))

    def save_model(self):
        state_dict = dict()
        state_dict["model_state_dict"] = self.model.state_dict()
        state_dict["optimizer_state_dict"] = self.optimizer.state_dict()
        state_dict["train_loss_history"] = torch.tensor([self.metric[i]["train_loss"] for i in range(self.train_epoch_idx)])
        state_dict["val_loss_history"] = torch.tensor([self.metric[i]["val loss"] for i in range(self.val_epoch_idx)])
        state_dict["pck_50"] = self.metric[self.val_epoch_idx]["pck_50"]
        state_dict["pck_40"] = self.metric[self.val_epoch_idx]["pck_40"]
        state_dict["pck_30"] = self.metric[self.val_epoch_idx]["pck_30"]
        state_dict["pck_20"] = self.metric[self.val_epoch_idx]["pck_20"]
        state_dict["pck_10"] = self.metric[self.val_epoch_idx]["pck_10"]
        state_dict["pck_5"] = self.metric[self.val_epoch_idx]["pck_5"]

        save_path = os.path.join(self.model_save_path, f"model_best.pth")
        torch.save(state_dict, save_path)


In [8]:
with open('/home/nxhoang/Work/HPE/src/model/configs/BO copy.yaml') as f:
  sweep_config = yaml.safe_load(f)

In [9]:
other_config = {'modality': 'wifi-csi',
                        'protocol': 'protocol1',
                        'data_unit': 'frame',
                        'random_split': {'ratio': 0.8,
                                         'random_seed': 42,
                                         'train_dataset': {'split': 'training',
                                                            'scenes': 'None',
                                                            'subjects': 'None',
                                                          'actions': 'all'},
                                          'val_dataset': {'split': 'validation',
                                                          'scenes': 'None',
                                                          'subjects': 'None',
                                                          'actions': 'all'}},
                        'cross_scene_split': {'train_dataset': {'split': 'training',
                                              'scenes': ['E01', 'E02', 'E03'],
                                              'subjects': 'None',
                                              'actions': 'all'},
                                              'val_dataset': {'split': 'validation',
                                                              'scenes': ['E04'],
                                                              'subjects': 'None',
                                                              'actions': 'all'}},
                        'cross_subject_split': {'train_dataset': {'split': 'training',
                                                                  'scenes': 'None',
                                                                  'subjects': ['S01','S02','S03','S04','S06','S07','S08','S09','S11','S12','S13','S14','S16','S17','S18','S19','S21','S22','S23','S24','S26','S27','S28','S29','S31','S32','S33','S34','S36','S37','S38','S39'],
                                                                  'actions': 'all'},
                                                'val_dataset': {'split': 'validation',
                                                                'scenes': 'None',
                                                                'subjects': ['S05', 'S10', 'S15', 'S20', 'S25', 'S30', 'S35', 'S40'],
                                                                'actions': 'all'}},
                        'manual_split': {'train_dataset': {'split': 'training',
                                                           'scenes': 'None',
                                                           'subjects': ['S01','S02','S03','S04','S05','S06','S07','S08','S09','S10','S11','S12','S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23','S24','S25','S26','S27','S28','S29','S30','S31','S32','S33','S34','S35','S36','S37','S38','S39','S40'],
                                                           'actions': ['A01','A02','A03','A04','A05','A06','A07','A08','A09','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19','A20','A21']},
                                          'val_dataset': {'split': 'validation',
                                                          'scenes': 'None',
                                                          'subjects': ['S01','S02','S03','S04','S05','S06','S07','S08','S09','S10','S11','S12','S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23','S24','S25','S26','S27','S28','S29','S30','S31','S32','S33','S34','S35','S36','S37','S38','S39','S40'],
                                                          'actions': ['A22', 'A23', 'A24', 'A25', 'A26', 'A27']}},
                        'split_to_use': 'random_split',
                        'init_rand_seed': 0,
                        'data_root': '/home/nxhoang/Work/HPE/Data',
                        'AE_checkpoint': '/home/nxhoang/Work/HPE/src/model/checkpoints/AE_model_best.pth',
                        'combined_checkpoint': '/home/nxhoang/Work/HPE/src/model/checkpoints/combined_model_best.pth'}

In [10]:
#training 
def training():
     with wandb.init(config=sweep_config):
        config = wandb.config
        
        # start = time.perf_counter()
        
    ## Create noise dataset
        train_dataset, test_dataset = make_dataset(other_config["data_root"], other_config) #train_dataset gom n mau, moi mau gom
        data_shape = train_dataset[0][1].shape
        # print("data_shape",)


        rng_generator = torch.manual_seed(other_config['init_rand_seed'])
        train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, batch_size = config['ae_batch_size'])
        val_data, test_data = train_test_split(test_dataset, test_size=0.5, random_state=41)
        val_loader = make_dataloader(val_data, is_training=False, generator=rng_generator, batch_size = config['ae_batch_size'])
        test_loader = make_dataloader(test_data, is_training=False, generator=rng_generator, batch_size = config['ae_batch_size'])

        # Define device
        device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

        #Initialize autoencoder denoiser
        denoiser = Denoiser(shape_data=data_shape, 
                            n_kernels=(config['ae_n_kernels_1'], config['ae_n_kernels_2']), 
                            kernel_size= [3,3], 
                            maxpooling=(2,  config['ae_maxpooling_2'])
                            ) 
        denoiser.to(device)


        # Train autoencoder denoiser 
        criterion_ae = nn.MSELoss().to(device)
        optimizer_ae = torch.optim.RMSprop(denoiser.parameters(), lr=config['ae_lr'], momentum=config['ae_momentum'])
        n_epochs_ae = config['ae_n_epochs']
        
        schedule_ae = LambdaLR(optimizer_ae, lr_lambda=lambda epoch: 1 if epoch < 40 else torch.exp(-0.1))
        trainer_ae = Denoise_Trainer(denoiser, train_loader, val_loader, test_loader, criterion=criterion_ae,
                                     optimizer=optimizer_ae, scheduler=schedule_ae)
        
        print("START TRAINING DENOISER AT: ", time.perf_counter_ns())
        for epoch in range(n_epochs_ae):
            trainer_ae.train_epoch()
            trainer_ae.val_epoch()
        print("END TRAINING DENOISER AT", time.perf_counter_ns)
        trainer_ae.test_epoch()


        # Extract Encoder part
        en_denoiser = denoiser.encoder
        en_denoiser.eval()
        for param in en_denoiser.parameters():
            param.requires_grad = False  # Freeze parameters

        #================================================ Initialize estimator ======================================#
        cnn_input_shape = (en_denoiser.n_kernels[-1], en_denoiser.h_pool2, en_denoiser.w_pool2)
        
        estimator = Estimator(shape_data=cnn_input_shape, 
                              dropout=config['est_dropout'],
                              n_basis_kernels=config['est_n_basis_kernels'],
                              kernel_size=(config['est_kernel_size_1'], config['est_kernel_size_2'], config['est_kernel_size_3'], config['est_kernel_size_4'], config['est_kernel_size_5']),
                              n_kernels=(config['est_n_kernels_1'], config['est_n_kernels_2'], config['est_n_kernels_3'],config['est_n_kernels_4'],config['est_n_kernels_5']),
                              num_layers=config['est_num_layers'],
                              pad=[1,1,1,1,1],
                              stride = [1,1,1,1,1],
                              maxPooling= (config['est_maxpooling_1'],  config['est_maxpooling_2'], config['est_maxpooling_3'],config['est_maxpooling_4'],config['est_maxpooling_5']),
                              temperature=config['est_temperature'], 
                              pool_dim='time',
                              d_linear = config['est_d_hidden']
                              )


        # Initialize a combined model
        combined_model = CombinedModel(en_denoiser, estimator).to(device)

        train_dataset1, test_dataset1 = original_make_dataset(other_config["data_root"], other_config)
        rng_generator1 = torch.manual_seed(other_config['init_rand_seed'])
        train_loader1 = original_make_dataloader(train_dataset1, is_training=True, generator=rng_generator1,
                                                 batch_size = config['est_batch_size'])
        val_data1, test_data1 = train_test_split(test_dataset1, test_size=0.5, random_state=41)
        val_loader1 = original_make_dataloader(val_data1, is_training=False, generator=rng_generator1,
                                               batch_size = config['est_batch_size'])
        test_loader1 = original_make_dataloader(test_data1, is_training=False, generator=rng_generator1,
                                                batch_size = config['est_batch_size'])

        # Training combined_model with Encoder parameters being frozen
        criterion_cb = nn.MSELoss().to(device)
        optimizer_cb = torch.optim.SGD(combined_model.parameters(), lr=config['cb_lr'], momentum=config['cb_momentum'])
        n_epochs_cb = config["cb_n_epochs"]
        n_epochs_decay = 60
        epoch_count = 1

        # def lambda_rule(epoch):
        #     lr_l = 1.0 - max(0, epoch + epoch_count - n_epochs_cb) / float(n_epochs_decay + 1)
        #     return lr_l

        scheduler_cb = torch.optim.lr_scheduler.LambdaLR(optimizer_cb, lr_lambda=lambda epoch: 1.0 - max(0,
                                                                                                         epoch + epoch_count - n_epochs_cb) / float(
            n_epochs_decay + 1))

        trainer_cb = Estimastor_Trainer(combined_model, train_loader1, val_loader1, test_loader1, criterion=criterion_cb,
                                        optimizer=optimizer_cb, scheduler=scheduler_cb)
        
        print("START TRAINING ESTIMATOR AT: ", time.perf_counter_ns())
        for epoch in range(n_epochs_cb):
            trainer_cb.train_epoch()
            trainer_cb.val_epoch()
        print("END TRAINING ESTIMATOR AT: ", time.perf_counter_ns())    
        trainer_cb.test_epoch()




        

In [11]:
sweep_id = wandb.sweep(sweep = sweep_config, project ="Denoising + HPE")

Create sweep with ID: 0n0fnj5c
Sweep URL: https://wandb.ai/nxhoang/Denoising%20%2B%20HPE/sweeps/0n0fnj5c


In [ ]:
!date
wandb.agent(sweep_id, function=training, count=100)

Thứ sáu, 05 Tháng 4 năm 2024 11:37:47 +07


wandb: Agent Starting Run: d4qyut03 with config:
wandb: 	ae_batch_size: 256
wandb: 	ae_lr: 0.00024312470709938943
wandb: 	ae_maxpooling_2: 2
wandb: 	ae_momentum: 0.5
wandb: 	ae_n_epochs: 3
wandb: 	ae_n_kernels_1: 64
wandb: 	ae_n_kernels_2: 76
wandb: 	cb_lr: 0.004479894776006358
wandb: 	cb_momentum: 0.9
wandb: 	cb_n_epochs: 100
wandb: 	est_batch_size: 512
wandb: 	est_d_hidden: 238
wandb: 	est_dropout: 0.37798698354639226
wandb: 	est_kernel_size_1: 3
wandb: 	est_kernel_size_2: 4
wandb: 	est_kernel_size_3: 2
wandb: 	est_kernel_size_4: 2
wandb: 	est_kernel_size_5: 4
wandb: 	est_maxpooling_1: 2
wandb: 	est_maxpooling_2: 3
wandb: 	est_maxpooling_3: 3
wandb: 	est_maxpooling_4: 3
wandb: 	est_maxpooling_5: 2
wandb: 	est_n_basis_kernels: 227
wandb: 	est_n_kernels_1: 32
wandb: 	est_n_kernels_2: 32
wandb: 	est_n_kernels_3: 111
wandb: 	est_n_kernels_4: 120
wandb: 	est_n_kernels_5: 95
wandb: 	est_num_layers: 2
wandb: 	est_temperature: 31


START TRAINING DENOISER AT:  9072295822361
Training Denoiser. Epoch: 0
Iter 0: MSE - 0.3353310525417328
Iter 1: MSE - 0.4120560884475708
Iter 2: MSE - 0.4031284749507904
Iter 3: MSE - 0.270698606967926
Iter 4: MSE - 0.21564409136772156
Iter 5: MSE - 0.14912065863609314
Iter 6: MSE - 0.11602136492729187
Iter 7: MSE - 0.09331948310136795
Iter 8: MSE - 0.08044205605983734
Iter 9: MSE - 0.07101825624704361
Iter 10: MSE - 0.06300655752420425
Iter 11: MSE - 0.057206474244594574
Iter 12: MSE - 0.052568063139915466
Iter 13: MSE - 0.046891726553440094
Iter 14: MSE - 0.0471171960234642
Iter 15: MSE - 0.04047223925590515
Iter 16: MSE - 0.037456121295690536
Iter 17: MSE - 0.040151212364435196
Iter 18: MSE - 0.07831579446792603
Iter 19: MSE - 0.09570695459842682
Iter 20: MSE - 0.05156398564577103
Iter 21: MSE - 0.043559156358242035
Iter 22: MSE - 0.04158908873796463
Iter 23: MSE - 0.03781091421842575
Iter 24: MSE - 0.03606032580137253
Iter 25: MSE - 0.033142805099487305
Iter 26: MSE - 0.03250371292

Iter 225: MSE - 0.0063533298671245575
Iter 226: MSE - 0.01193577703088522
Iter 227: MSE - 0.006284687668085098
Iter 228: MSE - 0.008068909868597984
Iter 229: MSE - 0.00655507342889905
Iter 230: MSE - 0.010992615483701229
Iter 231: MSE - 0.005540237762033939
Iter 232: MSE - 0.0056684184819459915
Iter 233: MSE - 0.00537072354927659
Iter 234: MSE - 0.005679743364453316
Iter 235: MSE - 0.006491797044873238
Iter 236: MSE - 0.00811736285686493
Iter 237: MSE - 0.005839212331920862
Iter 238: MSE - 0.005632407031953335
Iter 239: MSE - 0.005639456678181887
Iter 240: MSE - 0.005197499878704548
Iter 241: MSE - 0.006349379196763039
Iter 242: MSE - 0.006150806788355112
Iter 243: MSE - 0.012514147907495499
Iter 244: MSE - 0.006332147400826216
Iter 245: MSE - 0.008055731654167175
Iter 246: MSE - 0.008252739906311035
Iter 247: MSE - 0.007986086420714855
Iter 248: MSE - 0.005186665337532759
Iter 249: MSE - 0.005295461975038052
Iter 250: MSE - 0.0056028785184025764
Iter 251: MSE - 0.005717928521335125
It

Iter 447: MSE - 0.0046615502797067165
Iter 448: MSE - 0.005491113290190697
Iter 449: MSE - 0.0060577369295060635
Iter 450: MSE - 0.004270181525498629
Iter 451: MSE - 0.004214462358504534
Iter 452: MSE - 0.005297947209328413
Iter 453: MSE - 0.007015099748969078
Iter 454: MSE - 0.010996034368872643
Iter 455: MSE - 0.004781828261911869
Iter 456: MSE - 0.0048519023694098
Iter 457: MSE - 0.0037433281540870667
Iter 458: MSE - 0.0034459736198186874
Iter 459: MSE - 0.003722727531567216
Iter 460: MSE - 0.003648239653557539
Iter 461: MSE - 0.004855270031839609
Iter 462: MSE - 0.006302508991211653
Iter 463: MSE - 0.0037829000502824783
Iter 464: MSE - 0.004126880317926407
Iter 465: MSE - 0.006644500885158777
Iter 466: MSE - 0.00343719613738358
Iter 467: MSE - 0.00799487717449665
Iter 468: MSE - 0.005436688661575317
Iter 469: MSE - 0.008286584168672562
Iter 470: MSE - 0.0038160623516887426
Iter 471: MSE - 0.003993774764239788
Iter 472: MSE - 0.003667710581794381
Iter 473: MSE - 0.009363412857055664

...

100%|██████████| 65/65 [00:01<00:00, 45.48it/s]


START TRAINING ESTIMATOR AT:  66443911313940
Training Estimator.  Epoch 0


Traceback (most recent call last):
  File "<ipython-input-10-138c2b69e2a8>", line 106, in training
    trainer_cb.train_epoch()
  File "<ipython-input-7-d505aa21f10f>", line 124, in train_epoch
    predict = self.model(data)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-6-c93cdd4f1243>", line 12, in forward
    output = self.predictor(encoded)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-5-df92b821cd10>", line 105, in forward
    x = self.cnn(x)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py", line 204, in forward
    input = mo

Denoiser test result,▁
denoiser_train_loss,█▂▁
denoiser_val_loss,█▂▁
Denoiser test result,0.00243
denoiser_train_loss,0.00334
denoiser_val_loss,0.00258


Run svlq4qan errored:
Traceback (most recent call last):
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "<ipython-input-10-138c2b69e2a8>", line 106, in training
    trainer_cb.train_epoch()
  File "<ipython-input-7-d505aa21f10f>", line 124, in train_epoch
    predict = self.model(data)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-6-c93cdd4f1243>", line 12, in forward
    output = self.predictor(encoded)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-5-df92b821cd10>", line 105, in forward
    x = self.cnn(x)
  File "/home/nxhoang/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward